<a href="https://colab.research.google.com/github/Yoshiki0418/Deep_Learning/blob/main/DL_Lecture1/Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autograd(自動微分)

Pytorchでは計算グラフを自動的に構築し、勾配を逆方向に計算することができる。tensor生成時に、`requires_grad=True`を設定することで自動微分を有効にすることができる。計算後に、`.backward`を呼ぶことで自動的に勾配が計算される。

勾配情報は、`.grad`属性に累積されるが、末端ノードに対する勾配しか保存されない。中間ノードに対する勾配を保存したい場合は、当該tensorに対して、`.retain_grad()`を実行する必要がある。

In [13]:
import torch

x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)

z = y * torch.log(x) + torch.sin(y)

z.backward()

print(x.grad)
print(y.grad)

tensor(1.5000)
tensor(-0.2968)


In [19]:
x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)

# 中間ノードの勾配も取得
sum_xy = x + y
sum_xy.retain_grad()

z = sum_xy.pow(2)
z.backward()

print(x.grad)
print(y.grad)
print(sum_xy.grad)

tensor(10.)
tensor(10.)
tensor(10.)


## 勾配の計算を排除したい時

モデルの学習にはトレーニングを検証が必要であり、トレーニングの際はパラメータの最適化のために勾配の計算をする必要があるのだが、検証の時には勾配を計算する必要がない。ここで、勾配を使用しないのにも何も設定していないと自動的に勾配が計算されてしまう。これでは、計算リソースを無駄に使用してしまう原因となり、効率の悪い学習になってしまう。

この問題を解決するためには、`with torch.no_grad()`を用いることで、その中で計算する際には勾配が計算されないようにすることができる。

通常は、`model.eval()'でモデルを評価モードに設定した後に記述されることが多い。

```
# モデルを評価モードに設定
model.eval()

# 勾配計算を行わないコンテキスト内での推論
with torch.no_grad():
    output = model(input_data)
```

`model.eval()`では何が行われているのか？

* ドロップアウト (Dropout):
 * トレーニング時: 一部のユニットをランダムに無効にする（ドロップアウト率に従って）。
 * 評価時: すべてのユニットを使用する。ドロップアウトは無効になります。

* バッチ正規化 (Batch Normalization):
 * トレーニング時: ミニバッチごとに計算された平均と分散を使用して正規化します。
 * 評価時: トレーニング中に集計された移動平均と移動分散を使用して正規化します。

```
import torch.nn as nn

batch_norm = nn.BatchNorm1d(num_features=10)
model = nn.Sequential(
    nn.Linear(10, 10),
    batch_norm,
    nn.Linear(10, 1)
)

# トレーニングモード
model.train()
output_train = model(input_data)  # ミニバッチごとの平均と分散で正規化

# 評価モード
model.eval()
output_eval = model(input_data)  # 移動平均と移動分散で正規化
```

In [23]:
x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)


with torch.no_grad():
    z1 = y * torch.log(x) + torch.sin(y)

# 勾配が計算されない
z1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn